In [ ]:
!pip install transformers
!pip install langdetect
!pip install profanity-check

In [1]:
from transformers import pipeline
import glob
import os
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
ROOT_FOLDER = '/content/drive/Shareddrives/NLP2/Project'

In [4]:
full_df = pd.read_csv(os.path.join(ROOT_FOLDER, 'data', 'full_w_sent.csv'))
full_df

,author_id,clean_text,country,following_count,like_count,listed_count,period,quote_count,retweet_count,text,trans_text,tweet_count,tweet_created_at,tweet_id,user_created_at,user_verified,username,label,new_label
0,1.204746e+18,"Post ""lost control""you are liar.",Turkey,502.0,0.0,0.0,during,0.0,0.0,"@Jerusalem_Post ""lost control""you are liar.","Post ""lost control""you are liar.",279.0,2021-05-10T23:52:34.000Z,1.391904e+18,2019-12-11T12:52:13.000Z,0.0,semarukiye,NEGATIVE,NEGATIVE
1,2.824527e+08,Anyone notice the flights over Jerusalem just ...,Israel,1190.0,3.0,13.0,before,0.0,0.0,Anyone notice the flights over Jerusalem just ...,Anyone notice the flights over Jerusalem just ...,329091.0,2021-05-05T18:26:34.000Z,1.390010e+18,2011-04-15T07:20:02.000Z,0.0,27kislev5719,NEGATIVE,NEGATIVE
2,7.017483e+07,@ Yigal Arbiv IKMA Israel,Israel,2282.0,0.0,9.0,before,0.0,0.0,https://t.co/P8JE11Ce5m @ Yigal Arbiv IKMA Isr...,@ Yigal Arbiv IKMA Israel,5603.0,2021-05-04T12:25:03.000Z,1.389557e+18,2009-08-30T18:04:10.000Z,0.0,yigal289,POSITIVE,POSITIVE
3,1.141902e+18,Baru saja mengirim foto @ Palestine,Israel,45.0,1.0,0.0,after,0.0,0.0,Baru saja mengirim foto @ Palestine https://t....,Just post photos @ Palestine,1804.0,2021-05-30T09:36:53.000Z,1.398936e+18,2019-06-21T02:53:46.000Z,0.0,sudut50kota,NEGATIVE,NEGATIVE
4,1.750049e+08,This racist world doesn't care or gives a sh**...,Jordan,526.0,0.0,2.0,before,0.0,1.0,This racist world doesn't care or gives a sh*...,This racist world doesn't care or gives a sh**...,3009.0,2021-05-05T15:36:59.000Z,1.389970e+18,2010-08-05T11:14:33.000Z,0.0,mariabasem,NEGATIVE,NEGATIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48991,1.216493e+18,YA Kahhar GazzaUnderAttackk GazzaUnderAttack H...,Turkey,1777.0,3.0,1.0,during,0.0,0.0,YA Kahhar\n#GazzaUnderAttackk \n#GazzaUnderAtt...,Y Aka Ha RG AZ ZA Under Attack Kg Az Za Under ...,1448.0,2021-05-16T23:38:57.000Z,1.394075e+18,2020-01-12T22:52:45.000Z,0.0,ffeyzaorkc,NEGATIVE,NEGATIVE
48992,9.985172e+17,"At the end of the day, Finally the people of G...",Egypt,770.0,8.0,0.0,during,0.0,1.0,"At the end of the day, Finally the people of G...","At the end of the day, Finally the people of G...",583.0,2021-05-21T09:38:14.000Z,1.395675e+18,2018-05-21T10:53:45.000Z,0.0,Acc_Zhra_Agina,NEGATIVE,POSITIVE
48993,1.021264e+08,اللهم أهلك إsرائيل وجنودها اللهم مزقهم شر مُمَ...,Saudi Arabia,392.0,0.0,0.0,during,0.0,0.0,اللهم أهلك إsرائيل وجنودها\nاللهم مزقهم شر مُم...,"Oh God, your family and their soldiers,",1029.0,2021-05-15T21:05:33.000Z,1.393674e+18,2010-01-05T18:17:17.000Z,0.0,RonzaMalkawi,NEGATIVE,NEGATIVE
48994,2.510242e+08,UAE Canadian Ambassador: Sarah and Hagar are p...,United Arab Emirates,3505.0,31.0,196.0,after,1.0,8.0,UAE Canadian 🇨🇦Ambassador: Sarah and Hagar are...,UAE Canadian Ambassador: Sarah and Hagar are p...,15843.0,2021-06-02T18:55:35.000Z,1.400164e+18,2011-02-12T08:31:37.000Z,1.0,MarcyGrossman,POSITIVE,POSITIVE


In [11]:
sentiment_cls = pipeline('sentiment-analysis')

In [12]:
def get_sentiment(text):
  try:
    pred = sentiment_cls(text)
    label = pred[0]['label']
    return label
  except:
    return ''

In [ ]:
full_df['label'] = full_df['trans_text'].apply(get_sentiment)

In [ ]:
full_df  = full_df.dropna(subset=['trans_text'])

In [ ]:
opposite_words = ['el kuds', 'savesheikhjarrah', 'gazzaunderattack', 'freepalestine', 'savealaqsamosque']

In [ ]:
from nltk.tokenize import word_tokenize

def get_correct_sentiment(row):
  tokens = word_tokenize(row['trans_text'].lower())
  for token in tokens:
    if token in opposite_words:
      return 'NEGATIVE' if row['label'] == 'POSITIVE' else 'POSITIVE'
  return row['label']

In [ ]:
full_df['new_label'] = full_df.apply(lambda x: get_correct_sentiment(x), axis=1)

In [5]:
from langdetect import detect

langs = ['en', 'he', 'tr', 'ar']

def detect_lang(text):
  try:
    lg = detect(text)
    if lg in langs:
      return lg
    else:
      return 'other'
  except:
    return None

full_df['orig_lang'] = full_df['clean_text'].apply(detect_lang)

In [6]:
from profanity_check import predict, predict_prob

full_df['profanity_prob'] = full_df['trans_text'].apply(lambda c: predict_prob([c])[0])
full_df['profanity'] = full_df['profanity_prob'].apply(lambda x: True if x > 0.5 else False)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.20.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imp

In [ ]:
full_df.to_csv(ROOT_FOLDER + '/data/full_final.csv', index=False)